## 1- Importing Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## 2- Loading Dataset

In [2]:
def split2d(img, cell_size, flatten=True):
    h, w = img.shape[:2]
    sx, sy = cell_size
    cells = [np.hsplit(row, w//sx) for row in np.vsplit(img, h//sy)]
    cells = np.array(cells)
    if flatten:
        cells = cells.reshape(-1, sy, sx)
    return cells

def load_digits(file_name):
    digits_img = cv2.imread(file_name, 0)
    digits = split2d(digits_img, (20, 20))
    labels = np.repeat(np.arange(10), len(digits)/10)
    return digits, labels

digits, labels = load_digits('dataset/digits.png')
print(digits.shape)
print(labels.shape)

(5000, 20, 20)
(5000,)


## 3- Shuffle Dataset

In [3]:
rand = np.random.RandomState(10)
shuffle = rand.permutation(len(digits))

digits, labels = digits[shuffle], labels[shuffle]

## 4- Create a HOG object for feature extraction

In [4]:
winSize = (20,20)
blockSize = (8,8)
blockStride = (4,4)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64
signedGradient = True

hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradient)

## 5- Image feature extraction with HOG

In [5]:
hog_descriptors = []

for img in digits:
    hog_descriptors.append(hog.compute(img))
    
hog_descriptors = np.squeeze(hog_descriptors)
hog_descriptors.shape

(5000, 144)

## 6- Train/Test

In [6]:
train_n=int(0.9*len(hog_descriptors))

digits_train, digits_test = np.split(digits, [train_n])
hog_descriptors_train, hog_descriptors_test = np.split(hog_descriptors, [train_n])

labels_train, labels_test = np.split(labels, [train_n])

## 7- Creating SVM Model

In [7]:
C=12.5
gamma=0.50625
model = cv2.ml.SVM_create()

model.setGamma(gamma)
model.setC(C)
model.setKernel(cv2.ml.SVM_RBF)
model.setType(cv2.ml.SVM_C_SVC)

## 8- Training Model

In [8]:
model.train(hog_descriptors_train, cv2.ml.ROW_SAMPLE, labels_train)

True

## 9- Evaluation

In [9]:
predictions = model.predict(hog_descriptors_test)[1].ravel()

accuracy = (labels_test == predictions).mean()

print('Percentage Accuracy: %.2f %%' % (accuracy*100))

Percentage Accuracy: 98.60 %
